In [1]:
import pandas as pd
import json
import numpy as np

In [45]:
df = pd.read_json('dwts_scraper/scores.json')
df['dance_id'] = np.arange(df.shape[0])

In [46]:
df['judge'] = df['judge_phrase'].str.split(",", expand = False)

In [ ]:
# Some weeks (e.g., random have a `score` column instead of a `scores` column.  Rename that.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_20)#Weekly_scores Week 5.
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_17)#Weekly_scores Week 6.


# https://stackoverflow.com/questions/34989341/how-to-remove-nan-value-while-combining-two-column-in-panda-data-frame
df['scores'] = df['scores'].fillna(df['score'])

In [ ]:
https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_9)#Weekly_scores Week 6 has a 'ranked order' score dance.
Dance Offs are also a problem, some are separate tables, some are bottom rows in tables.






In [54]:
# Some weeks have a technical score/performance score. but no scores column.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Weekly_scores Week 4
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_10)#Weekly_scores Week 4

double_score = df[df['technical_score'].notna()]

double_score[double_score.columns[~double_score.isnull().any()]]

,couple,dance,music,result,season,week,judge_phrase,technical_score,performance_score,dance_id,judge
752,Kurt & Anna,Rumba,"""Drops of Jupiter (Tell Me)""—Train",Safe,11,Week 4: Acoustic Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","15 (5, 5, 5)","19 (7, 6, 6)",752,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
761,Erin & Maks,Tango,"""Sweet Dreams (Are Made of This)""—Eurythmics",Safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","18 (6, 6, 6)","21 (7, 7, 7)",761,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
871,Brandy & Maks,Rumba,"""This Woman's Work""—Kate Bush",Safe,11,Week 4: Acoustic Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","22 (7, 8, 7)","26 (9, 8, 9)",871,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
872,Rick & Cheryl,Argentine tango,"""Violentango""—Astor Piazzolla",Safe,11,Week 4: Acoustic Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","19 (6, 7, 6)","20 (6, 7, 7)",872,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
873,Kyle & Lacey,Rumba,"""Nothin' on You""—B.o.B feat. Bruno Mars",Safe,11,Week 4: Acoustic Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","18 (6, 6, 6)","22 (8, 7, 7)",873,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
874,The Situation & Karina,Argentine tango,"""Sweet Dreams (Are Made Of This)""—Tanghetto",Eliminated,11,Week 4: Acoustic Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","12 (4, 4, 4)","16 (6, 5, 5)",874,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
875,Florence & Corky,Rumba,"""Yesterday""—The Beatles",Safe,11,Week 4: Acoustic Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","17 (6, 6, 5)","18 (6, 6, 6)",875,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
881,Evan & Anna,Tango,"""Wait a Minute""—The Pussycat Dolls feat. Timba...",Safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","26 (9, 8, 9)","26 (9, 8, 9)",881,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
882,Niecy & Louis,Rumba,"""Taking Chances""—Celine Dion",Last to be called safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","18 (6, 6, 6)","18 (6, 6, 6)",882,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."
883,Aiden & Edyta,Rumba,"""Live Like We're Dying""—Kris Allen",Eliminated,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.","15 (5, 5, 5)","18 (6, 6, 6)",883,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio..."


In [5]:
new_cols = df['scores'].str.extract(r'(?P<total_score>\d+)\s\((?P<indiv_scores>.*)\)$')
new_cols['judge_score_list'] = new_cols['indiv_scores'].str.split(",")

new_cols[new_cols['judge_score_list'].isna()]

,total_score,indiv_scores,judge_score_list
226,NaN,NaN,NaN
227,NaN,NaN,NaN
402,NaN,NaN,NaN
540,NaN,NaN,NaN
549,NaN,NaN,NaN
...,...,...,...
2728,NaN,NaN,NaN
2791,NaN,NaN,NaN
2861,NaN,NaN,NaN
2862,NaN,NaN,NaN


In [27]:
# song_cols = df['music'].str.extract(r'"(?P<song_name>.*)"—(?P<song_artist>.*)$')
# song_cols
# df["music"].str.split(" / ") Some music columns have multiples.

In [6]:
df_new = pd.concat([df, new_cols], axis = 1)

In [7]:
( df_new.assign(judge_count = lambda df: df.judge.apply(len))
             .assign(score_count = lambda df: df.judge_score_list.apply(len))
)

TypeError: object of type 'float' has no len()

In [ ]:
df_final = ( df_new
              #  .head(5)
               .explode(['judge','judge_score_list'])
               .rename(columns={'judge_score_list': 'judge_score'})
               .drop(columns=['indiv_scores','judge_phrase','scores'])
               .assign(judge = lambda df: df.judge.str.strip(" ."))
)

In [ ]:
all_cols = df_final.columns.values.tolist()

front_cols = [ 'dance_id','season','week','couple','dance','music','judge','judge_score','total_score' ]

remaining_cols = [ele for ele in all_cols if ele not in front_cols]

reordered = front_cols + remaining_cols

In [ ]:
df_final = df_final[reordered]

In [ ]:
df_final